In [ ]:
import os
import shutil
import datetime
from ansys.edb.database import Database
from ansys.edb.geometry import ExtentType
from ansys.edb.session import launch_session
from ansys.edb.hierarchy.pin_group import PinGroup
from ansys.edb.terminal.terminals import PinGroupTerminal
from ansys.edb.terminal.terminals import BoundaryType

In [ ]:
"""EDB_RPC_SERVEUR.EXE path"""

RPC_SERVER = r"C:\Program Files\AnsysEM\v232\Win64"

In [ ]:
"""Copying local edb file"""

ROOT = os.getcwd()
session = launch_session(RPC_SERVER, 50051)
source_path = os.path.join(ROOT, "data", "edb", "Galileo_edb.aedb")
target_path = os.path.join(ROOT, "test_{}.aedb".format(str(datetime.datetime.now().date())))
shutil.copytree(source_path, target_path)

In [ ]:
"""Open EDB"""

edb = Database.open(target_path, False)

In [ ]:
"""EDB Layout """

layout = edb.circuit_cells[0].layout

In [ ]:
"""Component list to be included for the simulation"""

edb_components = [cmp for cmp in edb.circuit_cells[0].layout.groups if cmp.name in ["U2A5", "U1B5"]]

In [ ]:
"""Signal nets to be included"""

signal_nets = ["M_DQ<0>", "M_DQ<1>", "M_DQ<2>", "M_DQ<3>", "M_DQ<4>", "M_DQ<5>", "M_DQ<6>", "M_DQ<7>"]

In [ ]:
"""Reference nets, will be used to defined reference pin group terminals"""

reference_net = ["GND"]

In [ ]:
"""Creating pingroup terminals on each component based on net selection"""

for cmp in edb_components:
    ref_pins = [pin for pin in cmp.members if pin.net.name in reference_net]
    if ref_pins:
        ref_pin_group = PinGroup.create(layout, "{}_{}_ref".format(cmp.name, reference_net[0]), ref_pins)
        ref_pin_group_term = PinGroupTerminal.create(layout, "{}_{}_ref".format(cmp.name, reference_net[0]),
                                                         ref_pin_group, reference_net[0], False)
        ref_pin_group_term.is_circuit_port = True
        ref_pin_group_term.boundary_type = BoundaryType.PORT
        for signal_net in signal_nets:
            pins = [pin for pin in cmp.members if pin.net.name == signal_net]
            if pins:
                pin_group = PinGroup.create(layout, "{}_{}".format(cmp.name, signal_net), pins)
                if pin_group:
                    pin_group_term = PinGroupTerminal.create(layout, "{}_{}".format(cmp.name, signal_net), pin_group,
                                                            signal_net, False)
                    pin_group_term.is_circuit_port = True
                    pin_group_term.boundary_type = BoundaryType.PORT
                    pin_group_term.reference_terminal = ref_pin_group_term

In [ ]:
"""Computing conformal extend based on net selection"""

included_nets = [net for net in layout.nets if net.name in signal_nets]
clipped_net = [net for net in layout.nets if net.name in reference_net]
expanded_extent = layout.expanded_extent(
    nets=included_nets,
    extent=ExtentType.CONFORMING,
    expansion_factor=0.01,
    expansion_unitless=False,
    use_round_corner=True,
    num_increments=1,
)

In [ ]:
"""Clipping design"""

cutout_cell = layout.cell.cutout(
    included_nets=included_nets,
    clipped_nets=clipped_net,
    clipping_polygon=expanded_extent,
    clean_clipping=True,
)

In [ ]:
"""Deleting initial cell after clipping"""

edb.circuit_cells[0].delete()

In [ ]:
"""Save, close EDB and grpc session """

edb.save()
edb.close()
session.disconnect()